# LangChain Chaining Tutorial

This notebook demonstrates various chaining techniques in LangChain including:
- Simple Sequential Chains
- Sequential Chains with Multiple Inputs/Outputs
- Router Chains
- LCEL (LangChain Expression Language)

## Prerequisites
```bash
pip install langchain langchain-openai langchain-community
```

In [ ]:
# Import required libraries
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os

# Set your API key
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

# Initialize the LLM
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

## 1. Simple Sequential Chain

The simplest form of chaining where output of one chain becomes input of the next.

In [ ]:
# Chain 1: Generate a product name
first_prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# Chain 2: Generate a tagline for the company
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchy tagline for the following company: {company_name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combine chains
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

# Run the chain
result = simple_chain.run("eco-friendly water bottles")
print(f"\nFinal Result: {result}")

## 2. Sequential Chain with Multiple Inputs/Outputs

More complex chains that can handle multiple variables across steps.

In [ ]:
# Chain 1: Generate a synopsis
synopsis_template = """You are a playwright. Given the title and era of a play,
write a synopsis for that play.

Title: {title}
Era: {era}
Synopsis:"""
synopsis_prompt = PromptTemplate(input_variables=["title", "era"], template=synopsis_template)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt, output_key="synopsis")

# Chain 2: Write a review based on synopsis
review_template = """You are a theater critic. Given the synopsis of a play,
write a review for that play.

Synopsis:
{synopsis}

Review:"""
review_prompt = PromptTemplate(input_variables=["synopsis"], template=review_template)
review_chain = LLMChain(llm=llm, prompt=review_prompt, output_key="review")

# Combine chains
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["title", "era"],
    output_variables=["synopsis", "review"],
    verbose=True
)

# Run the chain
result = overall_chain({"title": "The AI Revolution", "era": "2024"})
print(f"\nSynopsis:\n{result['synopsis']}")
print(f"\nReview:\n{result['review']}")

## 3. Router Chain

Routes inputs to different chains based on content.

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# Define destination chains
physics_template = """You are a physics professor. Explain the following concept:
{input}"""

math_template = """You are a math professor. Solve the following problem:
{input}"""

history_template = """You are a history professor. Explain the following historical event:
{input}"""

# Create prompt infos
prompt_infos = [
    {"name": "physics", "description": "Good for physics questions", "prompt_template": physics_template},
    {"name": "math", "description": "Good for math problems", "prompt_template": math_template},
    {"name": "history", "description": "Good for history questions", "prompt_template": history_template}
]

# Create destination chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt = PromptTemplate(template=p_info["prompt_template"], input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# Create default chain
default_chain = LLMChain(llm=llm, prompt=PromptTemplate(
    template="{input}",
    input_variables=["input"]
))

# Create router chain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# Create multi-prompt chain
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

# Test with different inputs
print(chain.run("What is Newton's second law?"))
print("\n" + "="*50 + "\n")
print(chain.run("Solve: 2x + 5 = 15"))
print("\n" + "="*50 + "\n")
print(chain.run("What caused World War I?"))

## 4. LCEL (LangChain Expression Language)

Modern way of chaining using pipes (|) - more Pythonic and flexible.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create a simple LCEL chain
prompt = PromptTemplate(
    template="Tell me a joke about {topic}",
    input_variables=["topic"]
)

# Chain using LCEL
lcel_chain = prompt | llm | StrOutputParser()

# Run the chain
result = lcel_chain.invoke({"topic": "AI"})
print(result)

## 5. Advanced LCEL - Multi-step Chain

Building complex chains with LCEL.

In [ ]:
# Step 1: Generate topic
topic_prompt = PromptTemplate(
    template="Generate a random {subject} topic",
    input_variables=["subject"]
)

# Step 2: Create content about topic
content_prompt = PromptTemplate(
    template="Write a brief paragraph about: {topic}",
    input_variables=["topic"]
)

# Step 3: Summarize content
summary_prompt = PromptTemplate(
    template="Summarize this in one sentence: {content}",
    input_variables=["content"]
)

# Build the chain
advanced_chain = (
    {"subject": RunnablePassthrough()}
    | topic_prompt
    | llm
    | StrOutputParser()
    | {"topic": RunnablePassthrough()}
    | content_prompt
    | llm
    | StrOutputParser()
    | {"content": RunnablePassthrough()}
    | summary_prompt
    | llm
    | StrOutputParser()
)

# Run the chain
result = advanced_chain.invoke("technology")
print(f"Final Summary: {result}")

## 6. Parallel Chains with LCEL

Running multiple chains in parallel and combining results.

In [ ]:
from langchain_core.runnables import RunnableParallel

# Define multiple prompts
pros_prompt = PromptTemplate(
    template="List 3 pros of {topic}",
    input_variables=["topic"]
)

cons_prompt = PromptTemplate(
    template="List 3 cons of {topic}",
    input_variables=["topic"]
)

# Create parallel chains
parallel_chain = RunnableParallel(
    pros=pros_prompt | llm | StrOutputParser(),
    cons=cons_prompt | llm | StrOutputParser()
)

# Run parallel chains
result = parallel_chain.invoke({"topic": "remote work"})
print(f"Pros:\n{result['pros']}\n")
print(f"Cons:\n{result['cons']}")

## Conclusion

This notebook covered:
- **SimpleSequentialChain**: Single input/output chaining
- **SequentialChain**: Multiple inputs/outputs across steps
- **Router Chain**: Dynamic routing based on input
- **LCEL**: Modern, Pythonic chaining with pipes
- **Parallel Execution**: Running chains simultaneously

LCEL is now the recommended approach for building chains in LangChain due to its flexibility and ease of use.